<a href="https://colab.research.google.com/github/b-whitman/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/Ben_Whitman_214_assignment_regression_classification_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.
- [ ] Watch Aaron's [video #1](https://www.youtube.com/watch?v=pREaWFli-5I) (12 minutes) & [video #2](https://www.youtube.com/watch?v=bDQgVt4hFgY) (9 minutes) to learn about the mathematics of Logistic Regression.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [79]:
df.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [80]:
df.dtypes

Burrito            object
Date               object
Yelp              float64
Google            float64
Chips              object
Cost              float64
Hunger            float64
Mass (g)          float64
Density (g/mL)    float64
Length            float64
Circum            float64
Volume            float64
Tortilla          float64
Temp              float64
Meat              float64
Fillings          float64
Meat:filling      float64
Uniformity        float64
Salsa             float64
Synergy           float64
Wrap              float64
Unreliable         object
NonSD              object
Beef               object
Pico               object
Guac               object
Cheese             object
Fries              object
Sour cream         object
Pork               object
Chicken            object
Shrimp             object
Fish               object
Rice               object
Beans              object
Lettuce            object
Tomato             object
Bell peper         object
Carrots     

In [0]:
# Train-test-validate split

df['Date_dt'] = pd.to_datetime(df['Date'])

In [82]:
train = df[df['Date_dt'].dt.year <= 2016]
val = df[df['Date_dt'].dt.year == 2017]
test = df[df['Date_dt'].dt.year >= 2018]
print(train.shape, val.shape, test.shape, df.shape)

# All observations in the original df are accounted for- seems a decent spot check

(298, 60) (85, 60) (38, 60) (421, 60)


In [83]:
# Getting baselines

target = 'Great'
y_train = train[target]
y_train.value_counts(normalize=True)

False    0.590604
True     0.409396
Name: Great, dtype: float64

In [0]:
# Guess majority class for all predictions
# Most burritos are not great...?

majority_class = y_train.mode()[0]
y_pred = [majority_class] * len(y_train)

In [85]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

0.5906040268456376

In [86]:
y_val = val[target]
y_pred = [majority_class] * len(y_val)
accuracy_score(y_val, y_pred)

0.5529411764705883

In [96]:
# Clean up categorical columns.
# Replace X and x with 1
# Replace NaN with 0

import numpy as np

cat_cols = ['Chips', 'Unreliable',	'NonSD',	'Beef',	'Pico',	'Guac',	'Cheese',	'Fries',
            'Sour cream',	'Pork',	'Chicken',	'Shrimp',	'Fish',	'Rice',	'Beans',
            'Lettuce',	'Tomato',	'Bell peper',	'Carrots',	'Cabbage',	'Sauce',
            'Salsa.1',	'Cilantro',	'Onion',	'Taquito',	'Pineapple',	'Ham',
            'Chile relleno',	'Nopales',	'Lobster',	'Queso',	'Egg',
            'Mushroom',	'Bacon',	'Sushi',	'Avocado',	'Corn',	'Zucchini']
train[cat_cols] = train[cat_cols].replace({np.nan:0, 'x':1, 'X':1})

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [97]:
# Do the same for the val and test set

val[cat_cols] = val[cat_cols].replace({np.nan:0, 'x':1, 'X':1})
test[cat_cols] = test[cat_cols].replace({np.nan:0, 'x':1, 'X':1})

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [0]:
# # In numeric columns, fill NaN values with mean

# num_cols = ['Yelp', 'Google', 'Cost', 'Hunger', 'Mass (g)', 'Density (g/mL)',
#             'Length', 'Circum', 'Volume', 'Tortilla', 'Temp', 'Meat', 'Fillings',
#             'Meat:filling', 'Uniformity', 'Salsa', 'Synergy', 'Wrap']

# for col in num_cols:
#   value = train[col].mean()
#   train[col] = train[col].fillna(value)

In [0]:
# # Repeat on val and test set

# for col in num_cols:
#   value = val[col].mean()
#   val[col] = val[col].fillna(value)

In [0]:
# for col in num_cols:
#   value = test[col].mean()
#   test[col] = test[col].fillna(value)

In [98]:
train.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great,Date_dt
0,California,1/18/2016,3.5,4.2,0,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2016-01-18
1,California,1/24/2016,3.5,3.3,0,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2016-01-24
2,Carnitas,1/24/2016,NaN,NaN,0,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2016-01-24
3,Asada,1/24/2016,NaN,NaN,0,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2016-01-24
4,California,1/27/2016,4.0,3.8,1,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,0,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,True,2016-01-27


In [101]:
train.tail()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great,Date_dt
296,California,12/2/2016,4.0,4.3,0,5.65,3.0,NaN,NaN,19.5,22.0,0.75,4.0,1.5,2.0,3.0,4.2,4.0,3.0,2.0,4.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2016-12-02
297,Other,12/2/2016,NaN,NaN,0,5.49,3.0,NaN,NaN,19.0,20.5,0.64,4.5,5.0,2.0,2.0,2.5,3.5,3.0,2.5,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2016-12-02
298,California,12/10/2016,3.5,3.7,0,7.75,4.0,NaN,NaN,20.0,21.0,0.70,3.5,2.5,3.0,3.3,1.4,2.3,2.2,3.3,4.5,0,0,1,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2016-12-10
299,Asada,12/10/2016,NaN,NaN,0,7.75,4.0,NaN,NaN,19.5,21.0,0.68,4.0,4.5,2.0,2.0,3.5,3.5,2.0,2.0,4.0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2016-12-10
300,Other,12/15/2016,4.5,4.6,0,6.99,3.7,NaN,NaN,24.0,18.5,0.65,3.6,4.0,4.0,3.0,3.8,4.3,NaN,3.8,2.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2016-12-15


In [113]:
val.head()

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great,Date_dt
301,California,1/4/2017,NaN,NaN,0,6.60,NaN,NaN,NaN,23.0,20.5,0.77,4.0,4.5,4.0,3.5,3.5,5.0,1.5,3.50,4.5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2017-01-04
302,Other,1/4/2017,NaN,NaN,0,6.60,NaN,NaN,NaN,20.5,21.5,0.75,4.0,2.0,NaN,4.0,NaN,4.6,4.2,3.75,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2017-01-04
303,Other,1/7/2017,NaN,NaN,0,8.50,3.9,NaN,NaN,21.0,21.0,0.74,3.0,4.5,4.1,3.0,3.7,4.0,4.3,4.20,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2017-01-07
304,Other,1/7/2017,NaN,NaN,0,7.90,4.0,NaN,NaN,20.5,21.0,0.72,3.5,4.0,4.0,3.0,4.0,4.5,4.0,3.80,4.8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2017-01-07
305,Other,1/10/2017,NaN,NaN,0,4.99,3.5,NaN,NaN,18.5,22.5,0.75,2.5,4.5,3.0,2.5,3.0,3.0,2.0,2.00,4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,False,2017-01-10


In [120]:
# One practical joker seems to have put a word answer in what I was expecting to
# be a binary column
val['Chips'] = val['Chips'].replace({'Yes':1})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [137]:
# Use scikit-learn for logistic regression

from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='lbfgs')

features = ['Yelp',	'Google',	'Chips',	'Cost',	'Hunger', 'Length',	'Circum',
            'Volume',	'Tortilla',	'Temp',	'Meat',	'Fillings',	'Meat:filling',	
            'Uniformity']
          
X_train = train[features]
X_val = val[features]

from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)

log_reg.fit(X_train_imputed, y_train)

print('Validation Accuracy 1', log_reg.score(X_val_imputed, y_val))

Validation Accuracy 1 0.8235294117647058


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [155]:
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

log_reg = LogisticRegression(solver='lbfgs')

features = ['Yelp',	'Google',	'Chips',	'Cost',	'Hunger', 'Length',	'Circum',
            'Volume',	'Tortilla',	'Temp',	'Meat',	'Fillings',	'Meat:filling',	
            'Uniformity', 'Salsa',	'Synergy',	'Wrap']
          
X_train = train[features]
X_val = val[features]

from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_val_scaled = scaler.transform(X_val_imputed)

log_reg.fit(X_train_scaled, y_train)

print('Validation Accuracy 2', log_reg.score(X_val_scaled, y_val))

# We add about 5% more accuracy just by scaling our data and adding a few more features

Validation Accuracy 2 0.8705882352941177


In [146]:
# Let's try it with all features

from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

log_reg = LogisticRegression(solver='lbfgs')

# features = ['Yelp',	'Google',	'Chips',	'Cost',	'Hunger', 'Length',	'Circum',
#             'Volume',	'Tortilla',	'Temp',	'Meat',	'Fillings',	'Meat:filling',	
#             'Uniformity', 'Salsa',	'Synergy',	'Wrap', 'Unreliable',	'NonSD',	
#             'Beef', 'Guac',	'Cheese',	'Fries',
#             'Sour cream',	'Pork',	'Chicken',	'Shrimp',	'Fish',	'Rice',	'Beans',
#             'Lettuce',	'Tomato',	'Bell peper',	'Carrots',	'Cabbage',	'Sauce',
#             'Salsa.1',	'Cilantro',	'Onion',	'Taquito',	'Pineapple',	'Ham',
#             'Chile relleno',	'Nopales',	'Lobster',	'Queso',	'Egg',
#             'Mushroom',	'Bacon',	'Sushi',	'Avocado',	'Corn',	'Zucchini']
          
X_train = train[features]
X_val = val[features]

from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_val_scaled = scaler.transform(X_val_imputed)

log_reg.fit(X_train_scaled, y_train)

print('Validation Accuracy 3', log_reg.score(X_val_scaled, y_val))

# We actually lose some accuracy by including this many features

Validation Accuracy 3 0.7529411764705882


In [145]:
# ... but some comes back when we don't scale those features? Curious!

from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

log_reg = LogisticRegression(solver='lbfgs')

# features = ['Yelp',	'Google',	'Chips',	'Cost',	'Hunger', 'Length',	'Circum',
#             'Volume',	'Tortilla',	'Temp',	'Meat',	'Fillings',	'Meat:filling',	
#             'Uniformity', 'Salsa',	'Synergy',	'Wrap', 'Unreliable',	'NonSD',	
#             'Beef', 'Guac',	'Cheese',	'Fries',
#             'Sour cream',	'Pork',	'Chicken',	'Shrimp',	'Fish',	'Rice',	'Beans',
#             'Lettuce',	'Tomato',	'Bell peper',	'Carrots',	'Cabbage',	'Sauce',
#             'Salsa.1',	'Cilantro',	'Onion',	'Taquito',	'Pineapple',	'Ham',
#             'Chile relleno',	'Nopales',	'Lobster',	'Queso',	'Egg',
#             'Mushroom',	'Bacon',	'Sushi',	'Avocado',	'Corn',	'Zucchini']
          
X_train = train[features]
X_val = val[features]

from sklearn.impute import SimpleImputer
imputer = SimpleImputer()
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train_imputed)
# X_val_scaled = scaler.transform(X_val_imputed)

log_reg.fit(X_train_imputed, y_train)

print('Validation Accuracy 4', log_reg.score(X_val_imputed, y_val))

Validation Accuracy 4 0.8235294117647058


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [153]:
# Looks like the test set also has some wonky values...
# Gonna have to bend the rules a little bit to fix it.

test['Chips'] = test['Chips'].replace({'No':0})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [156]:
X_test = test[features]
y_test = test[target]
X_test_imputed = imputer.transform(X_test)
X_test_scaled = scaler.transform(X_test_imputed)

print('Test Accuracy:', log_reg.score(X_test_scaled, y_test))

Test Accuracy: 0.7631578947368421
